In [2]:

import psycopg2
import psycopg2.extras

#main code
import faiss
import concurrent.futures
import json
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_community.vectorstores import FAISS
from langchain_core.documents import Document
from langchain.llms import AzureOpenAI
from langchain.embeddings.openai import OpenAIEmbeddings
import numpy as np
import re
import time
import logging

# Configure logging
import pandas as pd
import json
import os
from langchain.llms import AzureOpenAI
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.chains import RetrievalQAWithSourcesChain
from langchain.prompts import PromptTemplate


# Initialize the connection to PostgreSQL



connection = psycopg2.connect(
    dbname="your_database_name",
    user="your_username",
    password="your_password",
    host="localhost",
    port="5432"
)



# initialize LLM and Embeddings LLM
os.environ["OPENAI_API_TYPE"] = "azure"
os.environ["OPENAI_API_VERSION"] = "2023-03-15-preview"
os.environ["OPENAI_API_BASE"] = "your_api_base"
os.environ["OPENAI_API_KEY"] = "your_api_key"
llm = AzureOpenAI(engine="llm-acelerator", model_name="gpt-3.5-turbo", temperature=0)
MODEL = "text-embedding-ada-002"
embedding = OpenAIEmbeddings(
                deployment="llm-acelerator-embedding",
                model=MODEL,
                )




read = pd.read_csv(r"C:\Users\atul.gupta\cleaned_new_pepsico.csv")

read

new_alis = read[['DB Field Name','Definition']].iloc[:3000]
new_alis['embedding'] = None
print(new_alis)



for index, row in new_alis.iterrows():
    field_dict = {}
    field_name = row['DB Field Name']
    definition = row['Definition']
    combined_text = f"{field_name}: {definition}"
    #print(field_dict)
    embeddings = embedding_model.embed_query(combined_text)
    new_alis.at[index, 'embedding'] = embeddings
    #print(embeddings)
new_alis


with connection.cursor() as cursor:
    for index,row in new_alis.iterrows():
        embedding_vector = row['embedding']
        #print(embedding_vector) 
        
        cursor.execute(
            """
            INSERT INTO test_3kop(db_field_name,definition,embedding)
            VALUES(%s,%s,%s)
            """,
            (row['DB Field Name'], row['Definition'], embedding_vector)
        )  
print("Done")
connection.commit()

c:\Users\atul.gupta\Anaconda3\lib\site-packages\langchain_community\llms\openai.py:254: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain_community.chat_models import ChatOpenAI`
  warnings.warn(
c:\Users\atul.gupta\Anaconda3\lib\site-packages\langchain_community\llms\openai.py:1075: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain_community.chat_models import ChatOpenAI`
  warnings.warn(


                               DB Field Name  \
0        HID_Form_Status__Regen_Ag__Projects   
1     HID_Form_Status__Regen_Ag__Agriculture   
2     HID_Form_Status__Regen_Ag__Commodities   
3                 HID_Forms_Completion_Check   
4                                  Thank_you   
...                                      ...   
2995         MaterialMarketingTargetTypeCode   
2996                                   APLAL   
2997                                   PLNNR   
2998                                   STLAN   
2999                                   STLAL   

                                             Definition embedding  
0     Calculated value of overall form completion fo...      None  
1     Calculated value of overall form completion fo...      None  
2     Calculated value of overall form completion fo...      None  
3     Calculated value of overall form completion fo...      None  
4     Message is typically displayed on the last pag...      None  
...            

In [4]:
import numpy as np
import ast

def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

def search_similar_definitions(query, top_n=5):
    query_embedding = embedding_model.embed_query(query)

    with connection.cursor() as cursor:
        cursor.execute("SELECT db_field_name, definition, embedding FROM test_3kop")
        results = cursor.fetchall()
        

    similarities = []
    for row in results:
        db_field_name, definition, embedding_str = row
        
        
        embedding = ast.literal_eval(embedding_str)  
        
        similarity = cosine_similarity(query_embedding, embedding)
        similarities.append((db_field_name, definition, similarity))

    
    similarities.sort(key=lambda x: x[2], reverse=True)
    return similarities[:top_n]
#query = "Footer"
query = ["Footer","asset_status","UpdatedById","MarketUnitName","JobFunctionName","PreferredName","GoLiveDate","CurrencyCode","PayrollEndDate","RequisitionId","DateOfDeath","ProbationPeriodEndDate","ResignationDate","ConfirmationDate","HireTypeCode","TerminationNotificationDate","CorporateCardNumber","ResignationTaiwanDate","LocationTypeCode","GeoZoneCode","PrimaryOwnerId","BusinessUnitName"]
for attribute in query:
    results = search_similar_definitions(attribute, top_n=5)
    print("Attribute_name:",attribute)
    for result in results:
        print(f"Field Name: {result[0]}, Definition: {result[1]}, Similarity: {result[2]}")
    print("\n")
    print("\n")
        


Attribute_name: Footer
Field Name: Footer, Definition: Captures information for survey flow Not used by PepsiCo for data capture, Similarity: 0.8134287823018188
Field Name: Footer, Definition: Captures survey form technical commands and logic Not used by PepsiCo for data capture, Similarity: 0.804000649129967
Field Name: XTNNoteToVendorText, Definition: Short note to supplier, Similarity: 0.792837764574322
Field Name: PoIntellectualPropertyRightsSection, Definition: Outlines measures to safeguard the buyer from claims of unauthorized use or infringement arising from the sale of the products or goods , Similarity: 0.7797965955109682
Field Name: XTNFashionThemeCode, Definition: The code specifies the fashion of the Theme, Similarity: 0.7784942554187625




Attribute_name: asset_status
Field Name: asset_status, Definition: Captures Survey form technical commands and logic Not used by PepsiCo for data capture, Similarity: 0.8370309110735907
Field Name: RecordStatusValue, Definition: Indica